# A collection of issues that I encountered working with CMIP6 models. Ill try to address them in PRs at some point.

In [6]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import intake
from cmip6_preprocessing.preprocessing import combined_preprocessing
from dask.diagnostics import ProgressBar
import cmocean.cm as cmo
import warnings

%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina'
xr.set_options(cmap_sequential='cividis', display_style='html')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
col = intake.open_esm_datastore("https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json")

## CM4 has unbelievably big chunks...this kills my workers immediately

I cant reproduce this right now...

In [26]:
# query = dict(variable_id=['o2'],
# #              experiment_id=['piControl'],
#              table_id=['Omon'], 
#              source_id=['GFDL-CM4'],
# #              grid_label=['gn']
#             )
# cat = col.search(**query)
# data_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times':True})
# # data_dict

## `combined_preprocessing` introduces duplicate values for nominal x and y

In [ ]:
# this should go to 

## ACCESS MODEL salinity fails to read



In [5]:
query = dict(variable_id=['so'],
             experiment_id=['historical'],
             table_id=['Omon'], 
             source_id=['ACCESS-ESM1-5'])
cat = col.search(**query)
data_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times':True})

Progress: |-------------------------------------------------------------------------------| 0.0% 

KeyError: 'cmip6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Omon/so/gn/.zmetadata'

## Time issues

I have seen several issue with decoding the time. Lets explore:

In [10]:

experiment_ids = ['historical']#,'ssp126',, 'ssp245', 'ssp585'
query = dict(variable_id=['thetao'],#'uo', 'vo', , 'o2'
             experiment_id=experiment_ids,
             table_id=['Omon'])
cat = col.search(**query)
models = cat.df['source_id'].unique()
print(models)
data_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times':True})
# So this fails with the preprocess argument, Ill have to see for which model exactly
for k,ds in data_dict.items():
    try:
        ds_new = combined_preprocessing(ds)
    except:
        print(f"{k} failed to preprocess")

['AWI-CM-1-1-MR' 'BCC-CSM2-MR' 'BCC-ESM1' 'CAMS-CSM1-0' 'FGOALS-f3-L'
 'CanESM5-CanOE' 'CanESM5' 'CNRM-CM6-1' 'CNRM-ESM2-1' 'ACCESS-CM2'
 'ACCESS-ESM1-5' 'E3SM-1-0' 'E3SM-1-1-ECA' 'E3SM-1-1' 'EC-Earth3-Veg'
 'EC-Earth3' 'FIO-ESM-2-0' 'INM-CM4-8' 'IPSL-CM6A-LR' 'MIROC-ES2L'
 'MIROC6' 'HadGEM3-GC31-LL' 'UKESM1-0-LL' 'MPI-ESM1-2-HR' 'MRI-ESM2-0'
 'GISS-E2-1-G-CC' 'GISS-E2-1-G' 'GISS-E2-1-H' 'CESM2-FV2'
 'CESM2-WACCM-FV2' 'CESM2-WACCM' 'CESM2' 'NorCPM1' 'GFDL-CM4' 'GFDL-ESM4'
 'NESM3' 'SAM0-UNICON' 'MCM-UA-1-0']
Progress: |███████████████████████████████████████████████████████████████████████████████| 100.0% 

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
             
--> There are 46 group(s)
CESM2-FV2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
CESM2-WACCM-FV2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
CES

In [18]:
# Huh this also works. So it seems that the preprocessing messes something up prior to aggregation. Is this due to time?
data_dict_new = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times':True}, preprocess=combined_preprocessing)

Progress: |█████████████████████████████--------------------------------------------------| 37.0% MIROC-ES2L: No units found
MIROC6: No units found
Progress: |██████████████████████████████████---------------------------------------------| 43.5% MIROC6: No units found
MIROC-ES2L: No units found
CESM2-FV2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
Progress: |█████████████████████████████████████████--------------------------------------| 52.2% CESM2-WACCM-FV2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
Progress: |█████████████████████████████████████████████----------------------------------| 56.5% CESM2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
MIROC6: No units found
CESM2-WACCM: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
MIROC-ES2L: No units found
Progress: |████████████████████████████████████████████████-------------------------------

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [16]:
# Checking each model to constrain the models that have problems, odly this works?
for model in models:
    print(f"+++++++{model}+++++++++")
    model_query = {k:v for k,v in query.items()}
    model_query['source_id'] = model
    model_cat = col.search(**model_query)
    print(model_cat.df)
    try:
        model_data_dict = model_cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times':False}, preprocess=combined_preprocessing)
    except Exception as e:
        print(f"{k} failed to preprocess")
        print(e)

+++++++AWI-CM-1-1-MR+++++++++
  activity_id institution_id      source_id experiment_id member_id table_id  \
0        CMIP            AWI  AWI-CM-1-1-MR    historical  r1i1p1f1     Omon   
1        CMIP            AWI  AWI-CM-1-1-MR    historical  r2i1p1f1     Omon   
2        CMIP            AWI  AWI-CM-1-1-MR    historical  r3i1p1f1     Omon   
3        CMIP            AWI  AWI-CM-1-1-MR    historical  r4i1p1f1     Omon   
4        CMIP            AWI  AWI-CM-1-1-MR    historical  r5i1p1f1     Omon   

  variable_id grid_label                                             zstore  \
0      thetao         gn  gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...   
1      thetao         gn  gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...   
2      thetao         gn  gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...   
3      thetao         gn  gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...   
4      thetao         gn  gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...   

   dcpp_init_y

In [22]:
possible_fuckers

['IPSL-CM6A-LR']

In [21]:
# Checking all models with one excluded? This takes forever but revealed the IPSL as the culprit...quel surprise...
possible_fuckers = []
for model in models:
    print(f"+++++++{model} excluded+++++++++")
    model_query = {k:v for k,v in query.items()}
    model_query['source_id'] = [m for m in models if m != model]
    model_cat = col.search(**model_query)
    try:
        model_data_dict = model_cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times':False}, preprocess=combined_preprocessing)
        possible_fuckers.append(model)
    except:
        pass


+++++++AWI-CM-1-1-MR excluded+++++++++
Progress: |███████████████████████--------------------------------------------------------| 28.9% MIROC-ES2L: No units found
Progress: |████████████████████████████---------------------------------------------------| 35.6% MIROC6: No units found
Progress: |████████████████████████████████-----------------------------------------------| 40.0% MIROC6: No units found
MIROC-ES2L: No units found
Progress: |█████████████████████████████████----------------------------------------------| 42.2% CESM2-FV2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
Progress: |█████████████████████████████████████------------------------------------------| 46.7% CESM2-WACCM: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
Progress: |███████████████████████████████████████----------------------------------------| 48.9% MIROC-ES2L: No units found
Progress: |████████████████████████████████████████-----------

In [19]:
# Huh this also works. So it seems that the preprocessing messes something up prior to aggregation. Is this due to time?
picked_models = [m for m in models if ('CanESM' not in m)]
data_dict_new = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times':True}, preprocess=combined_preprocessing)

Progress: |███████████████████████████----------------------------------------------------| 34.8% MIROC-ES2L: No units found
Progress: |█████████████████████████████--------------------------------------------------| 37.0% MIROC6: No units found
Progress: |██████████████████████████████████---------------------------------------------| 43.5% MIROC6: No units found
MIROC-ES2L: No units found
Progress: |████████████████████████████████████████---------------------------------------| 50.0% CESM2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
Progress: |█████████████████████████████████████████--------------------------------------| 52.2% CESM2-FV2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
Progress: |███████████████████████████████████████████------------------------------------| 54.3% CESM2-WACCM-FV2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`
Progress: |██████████████████████████

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

## Preprocessing errors for deriving nominal lon and lat values

## NASA dissic

In [ ]:
import xarray as xr
import gcsfs
gcs = gcsfs.GCSFileSystem(token='anon')
ds = xr.open_zarr(gcs.get_mapper('gs://cmip6/CMIP/NASA-GISS/GISS-E2-1-G-CC/historical/r1i1p1f1/Omon/dissic/gn/'), consolidated=True)